In [ ]:
#OPRO Code
#Also includes testing of APE and APO on our domain to see how they performed on our domain, didnt use them in the end

In [1]:
import numpy as np
import pandas as pd
import json
import os
import openai

openai_api_key = ''

#reading in data
df_cot = pd.read_csv("/Users/stevenslater/Desktop/FinalProject/Data/cotprompts.csv")
df_train = pd.read_csv("/Users/stevenslater/Desktop/FinalProject/Data/trainwindowsimproved.csv")
df_test = pd.read_csv("/Users/stevenslater/Desktop/FinalProject/Data/testwindowsimproved.csv")
#df_cot.head()

In [ ]:
df_test.head()

In [3]:
def ask_gpt(question):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": question}
            ]
        )

        answer = response['choices'][0]['message']['content']
        tokens_used = response['usage']['total_tokens']
        input_tokens = len(question.split()) 
        return answer, tokens_used, input_tokens

    except openai.error.OpenAIError as e:
        return f"Error: {str(e)}", 0, 0
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}", 0, 0

In [65]:
def get_template_ins(examples_df):
    
    metaPrompt = """Your task is to generate the instruction <INS>. Below are some previous instructions with their scores. The score is the f-1 score and ranges from 0 to 1.

Instruction: Review conversations to ascertain customer vulnerability by identifying explicit indications of substantial health issues affecting daily functionality, severe life disruptions such as the loss of a loved one or employment, any tangible signs of emotional or economic vulnerability, or distinct shortcomings in handling financial responsibilities or navigating digital platforms. If any such indicators are unmistakably present, reply with 'Yes'; otherwise, issue a 'No'. A clear-cut 'Yes' or 'No' answer is required solely based on these established criteria. Answer: <Yes/No>"
Score: 0.611

Instruction: Analyze the customer conversation, and allocate a 'Yes' if it directly references any underlying medical conditions, traumatic life changes, or explicit signs of financial or emotional struggle affecting daily activities, or if there are clear indications of the customer's limited ability or confidence in managing financial tasks, technology, or essential literacy. If no such indications are present, assign a 'No'. Provide only a concise 'Yes' or 'No' determination based on these guidelines. Answer: <Yes/No>"
Score: 0.646

Instruction: Review the dialogue to identify any explicit mentions of health conditions impacting daily functioning, recent significant life changes such as bereavement or unemployment, observable evidence of substantial emotional or financial hardship, or difficulty in understanding and handling financial or digital tasks. If any of these vulnerability indicators are present, then respond with 'Yes'; if none are observed, then respond with 'No'. A definitive binary 'Yes' or 'No' conclusion is required for classification.
Score: 0.658

Instruction: Evaluate the customer conversation and respond with 'Yes' if there's explicit mention of any health conditions, debilitating life events, evidence of financial or emotional strain indicating low resilience, or signals of limited understanding or confidence in managing their finances or everyday technology. Otherwise, respond with 'No'. A definitive 'Yes' or 'No' is required based on these stated factors.
Score: 0.668

Instruction: Review the customer's dialogue for definitive references to health complications impacting routine activities, marked life events like loss of employment or personal upheaval, tangible signs of severe emotional or financial distress, or explicit difficulties with fundamental financial, literacy, or digital skills. Confirm vulnerability with a 'Yes' if manifest; if absent, provide a 'No'. Emit an unambiguous 'Yes' or 'No' verdict accordingly.
Score: 0.703

Below are examples of the problems, we are classifying customers conversations as vulnerable or not vulnerable. Our definition of vulnerability refers to customers who, due to their personal circumstances, are especially susceptible to harm. All customers are at risk of becoming vulnerable and this risk is increased by characteristics of vulnerability related to 4 key drivers:
Health - health conditions or illnesses that affect ability to carry out day-to-day tasks.
Life events - life events such as bereavement, job loss or relationship breakdown.
Resilience - low ability to withstand financial or emotional shocks.
Capability - low knowledge of financial matters or low confidence in managing money (financial capability). Low capability in other relevant areas such as literacy, or digital skills
"""

    example_string = ""
    for _, example_row in examples_df.iterrows():
        formatted_utterance = example_row['conversation_chunks']
        #example_string += "\n" + formatted_utterance + "\n" 
        ins_text = "<INS>"

        example_string += "\n" + "Conversation: " + formatted_utterance + "\n" + ins_text + "\n" + "Answer: " + ("Yes" if example_row['labels'] == 1 else "No") + "\n"


    instruction = "\n Generate an instruction that is different from all the instructions <INS> above, and has a higher score than all the instructions <INS> above. The instruction should be concise, effective, and generally applicable to the classifcation task at hand. The instruction must contain that we need a Yes or No response.\n"

    prompt = metaPrompt + example_string + instruction
    

    #print(prompt)
    return prompt

In [35]:
def query_api_ins(num_examples):
    
    results = []
    total_input_tokens = 0
    total_output_tokens = 0

    prompt = get_template_ins(df_train.sample(num_examples))
    print(prompt)
        
    
    answer, tokens_used, input_tokens = ask_gpt(prompt)
    total_input_tokens += input_tokens
    total_output_tokens += tokens_used - input_tokens

        
    cost_per_1k_input_tokens = 0.01
    cost_per_1k_output_tokens = 0.03
    total_cost = (total_input_tokens / 1000 * cost_per_1k_input_tokens) + \
                 (total_output_tokens / 1000 * cost_per_1k_output_tokens)

    print(f"Total Cost: {total_cost}")
    
    return answer


In [ ]:
query_api_ins(12)

In [68]:
#TESTING PROMPT INSTRUCTION (We do this in Zero-shot learning environment)
def get_template_test(example):
    
    metaPrompt = """Our definition of vulnerability refers to customers who, due to their personal circumstances, are especially susceptible to harm. All customers are at risk of becoming vulnerable and this risk is increased by characteristics of vulnerability related to 4 key drivers.
    Health - health conditions or illnesses that affect ability to carry out day-to-day tasks.

    Life events - life events such as bereavement, job loss or relationship breakdown.

    Resilience - low ability to withstand financial or emotional shocks.

    Capability - low knowledge of financial matters or low confidence in managing money (financial capability). Low capability in other relevant areas such as literacy, or digital skills
    """

    conversation = "\n" + "Conversation: " + example + "\n"

    #This changes for every new instruction we test
    instruction = "\nExamine the customer dialogue and categorize it as 'Yes' if there are definitive indicators of health issues impacting routine tasks, significant life events that could substantially affect the customer's well-being, or clear evidence of low resilience or capability in dealing with financial, emotional, technological, or literacy challenges. If these stated indicators of vulnerability are not present, classify it as 'No'. A categorical 'Yes' or 'No' is essential, reflecting these specific vulnerability parameters.Answer: <Yes/No>\n"
    prompt = metaPrompt + conversation + instruction

    return prompt

In [77]:
def query_api_test(sampled_df):
    results = []
    total_input_tokens = 0
    total_output_tokens = 0

    
    for _, row in sampled_df.iterrows():
        
        
        utterance = row['conversation_chunks']
        prompt = get_template_test(utterance)
        
        answer, tokens_used, input_tokens = ask_gpt(prompt)
        total_input_tokens += input_tokens
        total_output_tokens += tokens_used - input_tokens

        
        answer = answer.rstrip('.').strip()

        
        answer_binary = 1 if answer.lower() == "yes" else 0
        
        #print(f"Utterance: {utterance}")
        #print(f"Actual Label: {row['labels']}")
        #print(f"Answer: {answer}")
        
        results.append({
            'Data': utterance,
            'Actual Label': row['labels'],
            'ChatGPT Label': answer_binary
        })
    
    #monitoring cost
    cost_per_1k_input_tokens = 0.01
    cost_per_1k_output_tokens = 0.03
    total_cost = (total_input_tokens / 1000 * cost_per_1k_input_tokens) + \
                 (total_output_tokens / 1000 * cost_per_1k_output_tokens)

    print(f"Total Cost: {total_cost}")
    
    results_df = pd.DataFrame(results)
    
    return results_df


In [ ]:
results = query_api_test(df_test)

In [ ]:
#METRICS 
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


actual_labels = results['Actual Label'].astype(int)  
predicted_labels = results['ChatGPT Label'].astype(int)  

accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

confusion = confusion_matrix(actual_labels, predicted_labels)


print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion)


In [ ]:
#THE BELOW CODE WAS USED TO TEST OTHER POSSIBLE PROMPT OPTIMISAITON BASELINES AND WAS NOT USED IN THE FINAL IMPLEMENTATION
#WE TESTED APE AND APO TO ASSESS THEIR PERFORMANCE BEFORE CONSTRUCTING EDIPROMPT


In [14]:
#APE
#TESTING INSTRUCTION
#Template Construction
def get_template_ins_ape(examples_df):
    
    metaPrompt = """Below are examples of the problems, we are classifying customers conversations as vulnerable or not vulnerable. Our definition of vulnerability refers to customers who, due to their personal circumstances, are especially susceptible to harm. All customers are at risk of becoming vulnerable and this risk is increased by characteristics of vulnerability related to 4 key drivers:
Health - health conditions or illnesses that affect ability to carry out day-to-day tasks.
Life events - life events such as bereavement, job loss or relationship breakdown.
Resilience - low ability to withstand financial or emotional shocks.
Capability - low knowledge of financial matters or low confidence in managing money (financial capability). Low capability in other relevant areas such as literacy, or digital skills
"""

    example_string = ""
    for _, example_row in examples_df.iterrows():
        # Use the correct row data access methods
        formatted_utterance = example_row['conversation_chunks']
        #example_string += "\n" + formatted_utterance + "\n" 
        ins_text = "<INS>"

        example_string += "\n" + "Conversation: " + formatted_utterance + "\n" + ins_text + "\n" + ("Yes" if example_row['labels'] == 1 else "No") + "\n"

    #For initial
    #instruction = """\nGenerate a new instruction <INS>. The instruction should be concise, effective, and generally applicable to the classifcation task at hand. The instruction must contain that we need a Yes or No response.\n"""

    #For semantic similar ins
    instruction = """\nHere is the current instruction: 'Determine if the customer is 'Vulnerable' (Yes) or 'Not Vulnerable' (No) by evaluating indications of vulnerability connected to Health, Life events, Resilience, and Capability. Provide a 'Yes' response only if there are explicit signs of vulnerability in any of these areas; if not, or in doubt, respond with 'No'. Your answer should be either 'Yes' or 'No'.\n
Generate a new instruction <INS> that is semantically similar to the current instruction. The instruction should be concise, effective, and generally applicable to the classifcation task at hand. The instruction must contain that we need a Yes or No response.\n"""

    prompt = metaPrompt + example_string + instruction
    

    
    return prompt

In [6]:
def query_api_ins_ape(num_examples):
    results = []
    total_input_tokens = 0
    total_output_tokens = 0

    prompt = get_template_ins_ape(df_train.sample(num_examples))
    #print(prompt)
        
    
    answer, tokens_used, input_tokens = ask_gpt(prompt)
    total_input_tokens += input_tokens
    total_output_tokens += tokens_used - input_tokens

        
  
    #monitoring cost
    cost_per_1k_input_tokens = 0.01
    cost_per_1k_output_tokens = 0.03
    total_cost = (total_input_tokens / 1000 * cost_per_1k_input_tokens) + \
                 (total_output_tokens / 1000 * cost_per_1k_output_tokens)

    print(f"Total Cost: {total_cost}")
    
    return answer


In [ ]:
query_api_ins_ape(12)

In [37]:
def query_api_test_ape(sampled_df):
    results = []
    total_input_tokens = 0
    total_output_tokens = 0

    for _, row in sampled_df.iterrows():
        
        
        utterance = row['conversation_chunks']
        prompt = get_template_test(utterance)
        #print(prompt)

        answer, tokens_used, input_tokens = ask_gpt(prompt)
        total_input_tokens += input_tokens
        total_output_tokens += tokens_used - input_tokens

        # Format answer
        answer = answer.rstrip('.').strip()
        answer_binary = 1 if answer.lower() == "yes" else 0
        
        #print(f"Utterance: {utterance}")
        #print(f"Actual Label: {row['labels']}")
        #print(f"Answer: {answer}")
        results.append({
            'Data': utterance,
            'Actual Label': row['labels'],
            'ChatGPT Label': answer_binary
        })
    
    #monitoring cost
    cost_per_1k_input_tokens = 0.01
    cost_per_1k_output_tokens = 0.03
    total_cost = (total_input_tokens / 1000 * cost_per_1k_input_tokens) + \
                 (total_output_tokens / 1000 * cost_per_1k_output_tokens)

    print(f"Total Cost: {total_cost}")
    
    results_df = pd.DataFrame(results)
    
    return results_df


In [ ]:
results = query_api_test_ape(df_test)

In [ ]:
#METRICS 
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

actual_labels = results['Actual Label'].astype(int)  
predicted_labels = results['ChatGPT Label'].astype(int)  

accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

confusion = confusion_matrix(actual_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion)


In [94]:
#APO - Asks LLM for text feedbakc on how to uodate and improve prompt
#TESTING INSTRUCTION
#Template Construction
def get_template_ins_apo(examples_df):
    
    metaPrompt = """Below are examples of the problems, we are classifying customers conversations as vulnerable or not vulnerable. Our definition of vulnerability refers to customers who, due to their personal circumstances, are especially susceptible to harm. All customers are at risk of becoming vulnerable and this risk is increased by characteristics of vulnerability related to 4 key drivers:
Health - health conditions or illnesses that affect ability to carry out day-to-day tasks.
Life events - life events such as bereavement, job loss or relationship breakdown.
Resilience - low ability to withstand financial or emotional shocks.
Capability - low knowledge of financial matters or low confidence in managing money (financial capability). Low capability in other relevant areas such as literacy, or digital skills
"""

    example_string = ""
    for _, example_row in examples_df.iterrows():
        formatted_utterance = example_row['conversation_chunks']
        #example_string += "\n" + formatted_utterance + "\n" 
        ins_text = "<INS>"

        example_string += "\n" + "Conversation: " + formatted_utterance + "\n" + ins_text + "\n" + ("Yes" if example_row['labels'] == 1 else "No") + "\n"

    
    instruction = """\nHere is the current instruction: 'Analyze each excerpt of customer conversation to identify any indicators of vulnerability, referring to the following drivers of vulnerability: Health (e.g., health conditions), Life events (e.g., bereavement), Resilience (e.g., financial or emotional shock endurance), and Capability (e.g., financial literacy, digital skills). Confirm the presence of vulnerability with 'Yes' only if there is explicit mention or strong implication of these issues. In absence of such evidence, or where information is insufficient to conclude vulnerability, respond with ‘No'. Base your judgment solely on the conversation content provided, without assuming additional context or providing explanations.\n
Please provide text feedback on how to update and improve the current instruction. The new instruction should be be concise, effective, and generally applicable to the classifcation task at hand.\n"""

    prompt = metaPrompt + example_string + instruction
    
    return prompt

In [72]:
def query_api_ins_apo(num_examples):
    results = []
    total_input_tokens = 0
    total_output_tokens = 0

    prompt = get_template_ins_apo(df_train.sample(num_examples))
    #print(prompt)
        
    answer, tokens_used, input_tokens = ask_gpt(prompt)
    total_input_tokens += input_tokens
    total_output_tokens += tokens_used - input_tokens

        
  
    #monitoring cost
    cost_per_1k_input_tokens = 0.01
    cost_per_1k_output_tokens = 0.03
    total_cost = (total_input_tokens / 1000 * cost_per_1k_input_tokens) + \
                 (total_output_tokens / 1000 * cost_per_1k_output_tokens)

    print(f"Total Cost: {total_cost}")
    
    return answer

In [ ]:
query_api_ins_apo(12)

In [104]:
def query_api_test_apo(sampled_df):
    results = []
    total_input_tokens = 0
    total_output_tokens = 0

    
    for _, row in sampled_df.iterrows():
        
        
        utterance = row['conversation_chunks']
        prompt = get_template_test(utterance)
        #print(prompt)
       
        answer, tokens_used, input_tokens = ask_gpt(prompt)
        total_input_tokens += input_tokens
        total_output_tokens += tokens_used - input_tokens

        
        answer = answer.rstrip('.').strip()

        
        answer_binary = 1 if answer.lower() == "yes" else 0
        
        #print(f"Utterance: {utterance}")
        #print(f"Actual Label: {row['labels']}")
        #print(f"Answer: {answer}")
        results.append({
            'Data': utterance,
            'Actual Label': row['labels'],
            'ChatGPT Label': answer_binary
        })
    
    
    cost_per_1k_input_tokens = 0.01
    cost_per_1k_output_tokens = 0.03
    total_cost = (total_input_tokens / 1000 * cost_per_1k_input_tokens) + \
                 (total_output_tokens / 1000 * cost_per_1k_output_tokens)

    print(f"Total Cost: {total_cost}")
    
    
    results_df = pd.DataFrame(results)
    
    return results_df


In [ ]:
results = query_api_test_apo(df_test)

In [ ]:
#METRICS 
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

actual_labels = results['Actual Label'].astype(int) 
predicted_labels = results['ChatGPT Label'].astype(int) 


accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)


confusion = confusion_matrix(actual_labels, predicted_labels)


print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion)
